In [1]:
import re
from rich.progress import track
from functools import cache

In [2]:
with open('input19.txt') as f:
    input = f.read().split('\n\n')

messages = input[1].splitlines()

In [3]:
def parse(rule):
    if m := re.match(r'"([a-z])"', rule):
        return m[1]
    rule = rule.split(' | ')
    rule = [[int(num) for num in r.split()] for r in rule]
    return rule

rules1 = input[0].splitlines()
rules1 = (re.match(r'(\d+): (.+)', r) for r in rules1)
rules1 = {int(r[1]): r[2] for r in rules1}
rules1 = {k: parse(v) for k, v in rules1.items()}

In [4]:
rules2 = rules1.copy()
rules2[8] = [[42], [42, 8]]
rules2[11] = [[42, 31], [42, 11, 31]]

In [5]:
@cache
def validate(message, r=0, rules=1):
    rule = rules1[r] if rules == 1 else rules2[r]

    if isinstance(rule, str):
        return message == rule
    
    for subrule in rule:
        if len(subrule) == 1:
            if validate(message, subrule[0], rules):
                return True
        elif len(subrule) == 2:
            for s in range(1, len(message)):
                if validate(message[:s], subrule[0], rules) \
                    and validate(message[s:], subrule[1], rules):
                    return True
        elif len(subrule) == 3:
            for s2 in range(2, len(message)):
                for s1 in range(1, s2):
                    if validate(message[:s1], subrule[0], rules) \
                        and validate(message[s1:s2], subrule[1], rules) \
                        and validate(message[s2:], subrule[2], rules):
                        return True
    return False


In [6]:
sum(validate(m) for m in track(messages))

265

In [7]:
sum(validate(m, rules=2) for m in track(messages))

394